# **Procura de Problemas Éticos em Issues**

In [ ]:
#%pip install pandas
#%pip install nltk
#%pip install ipywidgets


import pandas as pd
import os
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

## **Fazendo a leitura dos dados:**

In [2]:
issues = pd.read_csv("out/daggerfall-unity-issues-3.csv")
comentarios = pd.read_csv("out/daggerfall-unity-comentarios-3.csv")

# Concatenando o Titulo e a Descrição das issues
issues['Info'] = issues['TituloIssue'] + issues['DescricaoIssue']

# Convertendo todos os tipos para String
issues['Info'] = issues['Info'].astype(str).str.lower()
comentarios['Comentario'] = comentarios['Comentario'].astype(str).str.lower()

## **Tokenizando os Comentarios, Titulo e Descrição das Issues**

In [3]:
# Definindo as StopWords
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

def removerStopWords(palavras):
    return [palavra for palavra in palavras if palavra not in stopWords and palavra.isalpha()]

# Criando um tokenizer que considera expressões
expressoes = {'data governance', 'data protection','informed consent', 'lack of data', 'user data collection' 'business ethics','conflict of interest',
            'human agency', 'intellectual property','regulatory approaches', 'self-conception','professional ethics', 'work ethics','common goods','individual differences',
            'non-discrimination', 'non-maleficence','prevention of harm', 'quality of life', 'respect for human autonomy', 'retention and addiction', 'social justice',
            'speech issues', 'technical robustness','computer abuse', 'malicious use'}

expressaoRegular = r'\b(?:' + '|'.join(expressoes) + r')\b|\w+'
tokenizer = RegexpTokenizer(expressaoRegular)


# Tokenize dos comentarios das issues
comentariosToK = []
for i in range (comentarios['Comentario'].size):
    comentariosToK.append(removerStopWords(tokenizer.tokenize(comentarios.loc[i]['Comentario'])))
    removerStopWords(comentariosToK[i])

# Tokenize do campo Info
tituloDescricaoToK = []
for k in range(issues['Info'].size):
    tituloDescricaoToK.append(removerStopWords(tokenizer.tokenize(issues.loc[k]['Info'])))


[nltk_data] Downloading package stopwords to /home/zoega/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **Contando a ocorrência de problemas éticos:**

In [4]:
# Definindo os EthicalIssues
EthicalIssues = ['bias', 'data governance', 'data protection', 'encryption', 'informed consent', 'lack of data', 'monetization', 'openness', 'privacy', 'user data collection'
            'authorship', 'autonomy', 'beneficence', 'business ethics', 'commerce', 'compliance', 'confidentiality', 'conflict of interest', 'context', 'dependability',
            'fairness', 'human agency', 'intellectual property', 'oversight', 'regulatory approaches', 'responsibility', 'trust', 'trustworthiness', 'axiology', 'freedom',
            'self-conception', 'solidarity', 'utility', 'care', 'competence', 'professional ethics', 'work ethics', 'access', 'accessibility', 'common goods', 'dignity',
            'diversity', 'equality', 'equity', 'humanity', 'inclusiveness', 'individual differences', 'inequality', 'justice', 'non-discrimination', 'non-maleficence',
            'participation', 'plurality', 'prevention of harm', 'quality of life', 'respect for human autonomy', 'retention and addiction', 'social justice', 'sustainability',
            'unemployment', 'welfare', 'accountability', 'accuracy', 'anonymity', 'comprehensibility', 'consistency', 'contestability', 'explainability', 'explicability',
            'integrity', 'interpretability', 'liability', 'reliability', 'safety', 'security', 'speech issues', 'technical robustness', 'traceability', 'transparency',
            'usability', 'computer abuse', 'malicious use']

# Criando o DataFrame que guardará as ocorrências dos comentários
ocorrenciasComents = pd.DataFrame()
linhaInicial = 0 * len(EthicalIssues)
palavrasAntes = "" * len(EthicalIssues)
palavrasDepois = "" * len(EthicalIssues)

ocorrenciasComents['EthicalIssues'] = EthicalIssues
ocorrenciasComents['Ocorrencias'] = linhaInicial
ocorrenciasComents["PalavrasAntes"] = palavrasAntes
ocorrenciasComents["PalavrasDepois"] = palavrasDepois
ocorrenciasComents = ocorrenciasComents.set_index('EthicalIssues')
ocorrenciasComents

# Criando o DataFrame que guardará as ocorrências do título e descrição
ocorrenciasTituloDescricao = pd.DataFrame()
linhaInicial = 0 * len(EthicalIssues)
palavrasAntes = "" * len(EthicalIssues)
palavrasDepois = "" * len(EthicalIssues)

ocorrenciasTituloDescricao['EthicalIssues'] = EthicalIssues
ocorrenciasTituloDescricao['Ocorrencias'] = linhaInicial
ocorrenciasTituloDescricao["PalavrasAntes"] = palavrasAntes
ocorrenciasTituloDescricao["PalavrasDepois"] = palavrasDepois
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao.set_index('EthicalIssues')
ocorrenciasTituloDescricao

,Ocorrencias,PalavrasAntes,PalavrasDepois
EthicalIssues,,,
bias,0,,
data governance,0,,
data protection,0,,
encryption,0,,
informed consent,0,,
...,...,...,...
traceability,0,,
transparency,0,,
usability,0,,


In [5]:
for i in range(len(comentariosToK)):
    for j in range(len(comentariosToK[i])):
        
        if(comentariosToK[i][j] in EthicalIssues):
            
            ocorrenciasComents.loc[comentariosToK[i][j], 'Ocorrencias'] += 1
            
            if((j - 1) >= 0):
                if(ocorrenciasComents.loc[comentariosToK[i][j],'PalavrasAntes'] == ''):
                    ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasAntes'] = comentariosToK[i][j - 1]
                else:
                    ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasAntes'] += ('\n' + comentariosToK[i][j - 1])
            if((j + 1) <= (len(comentariosToK[i]) - 1)):
                if(ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasDepois'] == ''):
                    ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasDepois'] = comentariosToK[i][j + 1]
                else:
                    ocorrenciasComents.loc[comentariosToK[i][j], 'PalavrasDepois'] += ('\n' + comentariosToK[i][j + 1])


for i in range(len(tituloDescricaoToK)):
    for j in range(len(tituloDescricaoToK[i])):
        
        if(tituloDescricaoToK[i][j] in EthicalIssues):
            
            ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'Ocorrencias'] += 1
            
            if((j - 1) >= 0):
                if(ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j],'PalavrasAntes'] == ''):
                    ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasAntes'] = tituloDescricaoToK[i][j - 1]
                else:
                    ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasAntes'] += ('\n' + tituloDescricaoToK[i][j - 1])
            if((j + 1) <= (len(tituloDescricaoToK[i]) - 1)):
                if(ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasDepois'] == ''):
                    ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasDepois'] = tituloDescricaoToK[i][j + 1]
                else:
                    ocorrenciasTituloDescricao.loc[tituloDescricaoToK[i][j], 'PalavrasDepois'] += ('\n' + tituloDescricaoToK[i][j + 1])

# Isso aqui funcionou, porém ainda não consegui resolver os problemas de links e imagens.

## **Criando o DataFrame de Saída**

In [6]:
ocorrenciasComents = ocorrenciasComents[ocorrenciasComents['Ocorrencias'] > 0]
ocorrenciasComents

,Ocorrencias,PalavrasAntes,PalavrasDepois
EthicalIssues,,,
bias,4,blatant\nconfirmation\nmipmap\nset,replace\nsorry\nhand\nmip
commerce,1,work,messenger
context,42,new\ndistinguishes\nmusic\nmusic\nmusic\nmusic...,dfu\nprovide\nparameters\nchange\nanymore\nsto...
oversight,22,bad\nprobably\ndfu\napologize\nresolved\nunfor...,part\nsince\nfixed\nrealized\nhampering\nwante...
responsibility,6,two\nthink\ngiven\nwould\nwould\nhuge,serialization\nsupport\nreplacement\nobject\no...
trust,24,well\nmême\nyet\none\namount\nrefactor\none\nn...,able\nalways\ncourse\nwork\nknowledge\npeople\...
freedom,3,complete\nmod\ncomplete,custom\nlike\nmake
utility,33,daggerfallworkshop\ndaggerfallworkshop\ndagger...,assetinjection\ncontentreader\ncontentreader\n...
care,39,without\nfine\ndfu\nrequest\nreally\ntake\nthi...,relative\neither\napi\nfeature\nbug\nmerge\nre...


### **Exportando os resultados para um csv:**

In [7]:
ocorrenciasTituloDescricao = ocorrenciasTituloDescricao.reset_index()
ocorrenciasComents = ocorrenciasComents.reset_index()

caminhoPasta = 'out'
nomeArquivo = 'Comentarios.csv'
caminhoArquivo = os.path.join(caminhoPasta, nomeArquivo)

if not os.path.exists(caminhoPasta):
    os.makedirs(caminhoPasta)

ocorrenciasComents.to_csv(caminhoArquivo, sep=',', index=False, header=True, na_rep='N/A', encoding='utf-8')

caminhoPasta = 'out'
nomeArquivo = 'TituloDescricao.csv'
caminhoArquivo = os.path.join(caminhoPasta, nomeArquivo)

if not os.path.exists(caminhoPasta):
    os.makedirs(caminhoPasta)

ocorrenciasTituloDescricao.to_csv(caminhoArquivo, sep=',', index=False, header=True, na_rep='N/A', encoding='utf-8')

In [23]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from collections import Counter

# Exemplo de dados
data = {
    "problemasEticos": ["bias", "commerce", "context", "oversight", "responsibility", "trust", "freedom"],
    "ocorrencia": [4, 1, 42, 22, 6, 24, 3],
    "palavrasAntes": [
        ["blatant", "confirmation", "mipmap", "set"],
        ["work", "messenger"],
        ["new", "distinguishes", "music", "music", "music", "music"],
        ["bad", "probably", "dfu", "apologize", "resolved", "unforgivable"],
        ["two", "think", "given", "would", "huge"],
        ["well", "même", "yet", "one", "amount", "refactor"],
        ["complete", "mod", "complete"]
    ],
    "palavrasDepois": [
        ["replace", "sorry", "hand", "mip"],
        ["custom", "like", "make"],
        ["dfu", "provide", "parameters", "change", "anymore"],
        ["part", "since", "fixed", "realized", "hampering"],
        ["serialization", "support", "replacement", "object"],
        ["able", "always", "course", "work"],
        ["custom", "like", "make"]
    ]
}

# Converte para DataFrame
df = pd.DataFrame(data)

# Função para criar a tabela com linhas expansíveis e contagem de palavras
def criarTabelaExpansivel(df):
    displayWidgets = []
    
    for _, linha in df.iterrows():
        # Conta as palavras em 'palavrasAntes' e 'palavrasDepois'
        antes_counts = Counter(linha['palavrasAntes'])
        depois_counts = Counter(linha['palavrasDepois'])
        
        # Cria uma linha com os dados principais (não expansível)
        linha_widget = widgets.HBox([
            widgets.Label(value=str(linha["problemasEticos"]), layout= widgets.Layout(width="120px")),
            widgets.Label(value=str(linha["ocorrencia"]), layout= widgets.Layout(width="60px")),
            widgets.Label(value="Clique para expandir", layout= widgets.Layout(width="200px"))
        ])
        
        # Cria o conteúdo expansível com `palavrasAntes` e `palavrasDepois` e suas contagens
        widgetDetalhes = widgets.VBox([
            widgets.Label(value=f"Palavras Antes: {', '.join([f'{word} ({count})' for word, count in antes_counts.items()])}"),
            widgets.Label(value=f"Palavras Depois: {', '.join([f'{word} ({count})' for word, count in depois_counts.items()])}")
        ])
        
        # Torna o conteúdo expansível/retrátil
        expansao = widgets.Accordion(children=[widgetDetalhes])
        expansao.set_title(0, f"Detalhes de '{linha['problemasEticos']}'")
        
        # Agrupa a linha principal e o conteúdo expansível
        linha_box = widgets.VBox([linha_widget, expansao])
        displayWidgets.append(linha_box)
    
    # Exibe a tabela com todas as linhas e detalhes
    display(widgets.VBox(displayWidgets))

# Exibe a tabela expansível
criarTabelaExpansivel(df)